### 0. Load environment variables
#### Case 1) If you use OpenAI's ChatGPT, then you need store the following variables in '.env' file:

- 'OPENAI_API_KEY' = XXXXXXXXXXXXXXX
- 'model_name' = XXXXXXXXXXXXXXX

##### Case 2) If you use AzureOpenAI's ChatGPT, then you need store the following variables in '.env' file:

- 'OPENAI_API_TYPE' = XXXXXXXXXXXXXXX
- 'OPENAI_API_VERSION' = XXXXXXXXXXXXXXX
- 'OPENAI_API_BASE' = XXXXXXXXXXXXXXX
- 'OPENAI_API_KEY' = XXXXXXXXXXXXXXX
- 'deployment_name' = XXXXXXXXXXXXXXX

In [1]:
import os
import sys  
from dotenv import load_dotenv
sys.path.extend(["..", '../..'])  
_ = load_dotenv('.env')


model_name = os.getenv('deployment_name') if os.getenv('OPENAI_API_TYPE') == 'azure' else os.getenv('model_name')
if os.getenv('OPENAI_API_TYPE') == 'azure':
    deployment_name = model_name

### 1. Get tools and tool descriptions

In [2]:
from utils.agent_components.get_llm import get_base_llm
from utils.agent_components.configurations import Configurations
config = Configurations(provider='AzureChatOpenAI')


from utils.agent_tools.tools.web_search_tools import get_web_search_tools
from utils.agent_tools.tools.weather_tools import GetFromOpenWeatherMap
from utils.agent_tools.tools.time_tools import GetFromDatetimeModule
from utils.agent_tools.tools.python_repl_tools import GetLangChainPythonRepl

search_tool = get_web_search_tools(config)
weather_tool = GetFromOpenWeatherMap()
time_tool = GetFromDatetimeModule()
python_repl_tool = GetLangChainPythonRepl()
tools = [search_tool, weather_tool, time_tool, python_repl_tool]

from utils.agent_tools.render.tools_render import My_render_text_description_and_args
tool_descriotions = My_render_text_description_and_args(tools)
tool_descriotions

"tool_1: Googling_from_SerpAPI: This is useful for when you need to search Google to answer questions. You should ask targeted questions. Input should be a single string not a list. Never pass in ['string', $query], Just pass in $query., args: {'tool_input': {'type': 'string'}}\ntool_2: GetFromOpenWeatherMap: Return the weather information based on current location. Not only can you get the current weather, you can also get weather prediction up to 7 days in advance. This function is two-input function. Input 1: 'IANA_timezone' is the timezone name of the current location with respect to IANA Time Zone Databasse. The default value is 'Asia/Seoul' (str). Input 2: 'Days_from_now', is the number of days from today. The default value is 0 (int). value is string. Only use this function for the current weather and weather forecast for up to 7 days. This functon CANNOT provide historical weather data., args: {'IANA_timezone': {'title': 'Iana Timezone', 'description': 'the region you want to k

### 2. Fetch the prompt and create a chain (using LCEL)

In [6]:
from langchain import hub
from langchain.chat_models import AzureChatOpenAI, ChatOpenAI
from langchain.schema.output_parser import StrOutputParser

prompt = hub.pull("jet-taekyo-lee/agent-task-generation")
deployment_name = os.getenv('deployment_name')


chat_model = AzureChatOpenAI(deployment_name=deployment_name, temperature=0.7) if os.getenv('OPENAI_API_TYPE') == 'azure' else ChatOpenAI(model_name=model_name, temperature=0.7)
task_generation_chain = prompt | chat_model | StrOutputParser()


### 3. Get a response 

In [7]:
answer = task_generation_chain.invoke({'tool_descriotions':tool_descriotions, 'num_tasks':100, 'h':3})
answer

'{"task": "Find out the current temperature in New York City."}\n{"task": "What is the current time in Sydney, Australia?"}\n{"task": "Perform a Google search to find the highest grossing film of all time."}\n{"task": "Execute the Python command \'print(5 + 7)\' and provide the output."}\n{"task": "Forecast the weather in London, UK, two days from now."}\n{"task": "What is the current date and time in Tokyo, Japan?"}\n{"task": "Perform a Google search to find the current President of France."}\n{"task": "Execute the Python command \'print([i*2 for i in range(10)])\' and provide the output."}\n{"task": "Find out the weather in Berlin, Germany, three days from now."}\n{"task": "What is the current time in Los Angeles, USA?"}\n{"task": "Perform a Google search to find the capital city of Brazil."}\n{"task": "Execute the Python command \'print(sum(range(1, 101)))\' and provide the output."}\n{"task": "Forecast the weather in Rome, Italy, five days from now."}\n{"task": "What is the current

### 4. Save outpus as jsonl object

In [8]:
from utils.post_processing.save_outputs_as_files import save_as_jsonl_files
save_as_jsonl_files(answer, file_name='temp_0_7.jsonl')